# ETL – Student Performance & Engagement Analytics  
## Dataset: Open University Learning Analytics Dataset (OULAD)

## 0) Basic setup

In [38]:
from __future__ import annotations

import os
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parent

DATA_RAW = PROJECT_ROOT / "data_raw"
DATA_CLEANED = PROJECT_ROOT / "data_cleaned"
DATA_DICT = PROJECT_ROOT / "data_dictionary"

DATA_CLEANED.mkdir(parents=True, exist_ok=True)
DATA_DICT.mkdir(parents=True, exist_ok=True)

## 1) EXTRACT – Load raw tables

In [39]:
students_raw = pd.read_csv(DATA_RAW / "studentInfo.csv")
courses_raw = pd.read_csv(DATA_RAW / "courses.csv")
assessments_raw = pd.read_csv(DATA_RAW / "assessments.csv")
student_assessment_raw = pd.read_csv(DATA_RAW / "studentAssessment.csv")
student_vle_raw = pd.read_csv(DATA_RAW / "studentVle.csv")

## 2) Validate columns

In [40]:
def require_cols(df: pd.DataFrame, cols: list[str], name: str) -> None:
    missing_cols = [c for c in cols if c not in df.columns]
    if missing_cols:
        raise ValueError(f"{name} is missing columns: {missing_cols}")

require_cols(
    students_raw,
    ["code_module", "code_presentation", "id_student", "gender", "region",
     "highest_education", "imd_band", "age_band", "num_of_prev_attempts",
     "studied_credits", "disability", "final_result"],
    "studentInfo.csv"
 )
require_cols(
    courses_raw,
    ["code_module", "code_presentation", "module_presentation_length"],
    "courses.csv"
 )
require_cols(
    assessments_raw,
    ["code_module", "code_presentation", "id_assessment", "assessment_type", "date", "weight"],
    "assessments.csv"
 )
require_cols(
    student_assessment_raw,
    ["id_assessment", "id_student", "date_submitted", "is_banked", "score"],
    "studentAssessment.csv"
 )
require_cols(
    student_vle_raw,
    ["code_module", "code_presentation", "id_student", "id_site", "date", "sum_click"],
    "studentVle.csv"
 )

## 3) CLEAN – Standardize types

In [41]:
def to_int(df: pd.DataFrame, col: str, name: str) -> pd.Series:
    # convert values to numbers; invalid -> NaN
    s = pd.to_numeric(df[col], errors="coerce")
    if s.isna().any():
        pass
    return s

# Students
students = students_raw.copy()
students["id_student"] = to_int(students, "id_student", "students").astype("Int64")
students["num_of_prev_attempts"] = to_int(students, "num_of_prev_attempts", "students").astype("Int64")
students["studied_credits"] = to_int(students, "studied_credits", "students").astype("Int64")

# Courses
courses = courses_raw.copy()
courses["module_presentation_length"] = to_int(courses, "module_presentation_length", "courses").astype("Int64")

# Assessments
assessments = assessments_raw.copy()
assessments["id_assessment"] = to_int(assessments, "id_assessment", "assessments").astype("Int64")
assessments["date"] = to_int(assessments, "date", "assessments").astype("Int64")
assessments["weight"] = pd.to_numeric(assessments["weight"], errors="coerce")

# Student Assessment
student_assessment = student_assessment_raw.copy()
student_assessment["id_assessment"] = to_int(student_assessment, "id_assessment", "student_assessment").astype("Int64")
student_assessment["id_student"] = to_int(student_assessment, "id_student", "student_assessment").astype("Int64")
student_assessment["date_submitted"] = to_int(student_assessment, "date_submitted", "student_assessment").astype("Int64")
student_assessment["is_banked"] = to_int(student_assessment, "is_banked", "student_assessment").astype("Int64")
student_assessment["score"] = pd.to_numeric(student_assessment["score"], errors="coerce")

# Student VLE
student_vle = student_vle_raw.copy()
student_vle["id_student"] = to_int(student_vle, "id_student", "student_vle").astype("Int64")
student_vle["id_site"] = to_int(student_vle, "id_site", "student_vle").astype("Int64")
student_vle["date"] = to_int(student_vle, "date", "student_vle").astype("Int64")
student_vle["sum_click"] = to_int(student_vle, "sum_click", "student_vle").astype("Int64")

## 4) CLEAN – Handle missing values

In [42]:
# Fill missing values with 'Unknown'
for col in ["imd_band", "highest_education", "region", "age_band", "gender", "disability"]:
    students[col] = students[col].astype("string")
    students[col] = students[col].fillna("Unknown")

students["final_result"] = students["final_result"].astype("string").fillna("Unknown")

# Drop rows missing key fields
students = students.dropna(subset=["id_student", "code_module", "code_presentation"])

courses = courses.dropna(subset=["code_module", "code_presentation"])

assessments = assessments.dropna(subset=["code_module", "code_presentation", "id_assessment", "weight"])
student_assessment = student_assessment.dropna(subset=["id_student", "id_assessment", "score"])
student_vle = student_vle.dropna(subset=["id_student", "code_module", "code_presentation", "sum_click", "date"])

## 5) CLEAN – Outliers / invalid values

In [43]:
# Assessment scores 0–100
student_assessment = student_assessment[(student_assessment["score"] >= 0) & (student_assessment["score"] <= 100)]

# Clicks cannot be negative
student_vle = student_vle[student_vle["sum_click"] >= 0]

# Remove assessments with weight <= 0
assessments = assessments[assessments["weight"] > 0]

## 6) CLEAN – Duplicates

In [44]:
students = students.drop_duplicates()
courses = courses.drop_duplicates()
assessments = assessments.drop_duplicates()
student_assessment = student_assessment.drop_duplicates()
student_vle = student_vle.drop_duplicates()

## 7) TRANSFORM – Build dimensions

In [45]:
# Student dimension: one row per student
# Use common values (mode) for categories; max for numbers
def mode_or_unknown(s: pd.Series) -> str:
    s = s.dropna()
    if s.empty:
        return "Unknown"
    m = s.mode(dropna=True)
    return str(m.iloc[0]) if not m.empty else "Unknown"

dim_students = (
    students.groupby("id_student", as_index=False)
    .agg(
        gender=("gender", mode_or_unknown),
        region=("region", mode_or_unknown),
        highest_education=("highest_education", mode_or_unknown),
        imd_band=("imd_band", mode_or_unknown),
        age_band=("age_band", mode_or_unknown),
        disability=("disability", mode_or_unknown),
        studied_credits=("studied_credits", "max"),
        num_of_prev_attempts=("num_of_prev_attempts", "max"),
        final_result=("final_result", mode_or_unknown),
    )
)

# Courses dimension
dim_courses = courses.rename(columns={
    "code_module": "module_code",
    "code_presentation": "presentation",
}).copy()

# Create course key
dim_courses["course_key"] = (dim_courses["module_code"].astype(str) + "_" + dim_courses["presentation"].astype(str))

# Assessments dimension
dim_assessments = assessments.rename(columns={
    "code_module": "module_code",
    "code_presentation": "presentation",
}).copy()
dim_assessments["course_key"] = (dim_assessments["module_code"].astype(str) + "_" + dim_assessments["presentation"].astype(str))

# Time dimension from activity and submissions
# Day indexes since course start
time_days = pd.Series(
    pd.concat([student_vle["date"], student_assessment["date_submitted"]], ignore_index=True).dropna().unique()
).sort_values()

dim_time = pd.DataFrame({"day_index": time_days.astype("int64")})
dim_time["week_index"] = (dim_time["day_index"] // 7).astype("int64")
dim_time["month_index_approx"] = (dim_time["day_index"] // 30).astype("int64")

## 8) TRANSFORM – Fact tables

In [46]:
# Performance fact: student x course
# Join to get weights and due dates
sa_join = student_assessment.merge(
    dim_assessments[["id_assessment", "module_code", "presentation", "course_key", "assessment_type", "weight", "date"]],
    on="id_assessment",
    how="inner"
)

# Weighted points = score * weight
sa_join["weighted_points"] = sa_join["score"] * sa_join["weight"]

# Group by student and course
fact_student_course_performance = (
    sa_join.groupby(["id_student", "course_key"], as_index=False)
    .agg(
        avg_score=("score", "mean"),
        assessments_taken=("score", "count"),
        total_weight=("weight", "sum"),
        weighted_points=("weighted_points", "sum"),
        last_submission_day=("date_submitted", "max"),
    )
)

# Weighted average score
fact_student_course_performance["weighted_avg_score"] = (
    fact_student_course_performance["weighted_points"] / fact_student_course_performance["total_weight"]
)

# Pass flag (>= 50)
fact_student_course_performance["pass_flag"] = fact_student_course_performance["weighted_avg_score"] >= 50

# At-Risk: score < 50 or few assessments
fact_student_course_performance["at_risk_flag"] = (
    (fact_student_course_performance["weighted_avg_score"] < 50) |
    (fact_student_course_performance["assessments_taken"] < 2)
)

# Engagement fact: student x course
vle_base = student_vle.rename(columns={
    "code_module": "module_code",
    "code_presentation": "presentation",
}).copy()
vle_base["course_key"] = (vle_base["module_code"].astype(str) + "_" + vle_base["presentation"].astype(str))

fact_student_course_engagement = (
    vle_base.groupby(["id_student", "course_key"], as_index=False)
    .agg(
        total_clicks=("sum_click", "sum"),
        active_days=("date", "nunique"),
        first_activity_day=("date", "min"),
        last_activity_day=("date", "max"),
    )
)

# Engagement score: log(clicks + 1) + active_days
# Use log to reduce skew
fact_student_course_engagement["engagement_score"] = (
    (fact_student_course_engagement["total_clicks"] + 1).apply(lambda x: float(pd.Series([x]).apply("log").iloc[0])) +
    fact_student_course_engagement["active_days"].astype(float)
)

# Integrated fact (merge performance and engagement)
fact_student_course = fact_student_course_performance.merge(
    fact_student_course_engagement,
    on=["id_student", "course_key"],
    how="left"
)

# Add course info
fact_student_course = fact_student_course.merge(
    dim_courses[["course_key", "module_code", "presentation", "module_presentation_length"]],
    on="course_key",
    how="left"
)

## 9) FINAL CLEAN – Ensure no weird infinities / NaNs in key metrics

In [47]:
# Clean up metrics and fill engagement gaps
fact_student_course["weighted_avg_score"] = pd.to_numeric(fact_student_course["weighted_avg_score"], errors="coerce")
fact_student_course = fact_student_course.dropna(subset=["id_student", "course_key", "weighted_avg_score"])

# Fill engagement fields for students with no VLE activity
for col in ["total_clicks", "active_days", "first_activity_day", "last_activity_day", "engagement_score"]:
    if col in fact_student_course.columns:
        fact_student_course[col] = fact_student_course[col].fillna(0)

## 10) LOAD – Save cleaned outputs

In [48]:
dim_students.to_csv(DATA_CLEANED / "dim_students.csv", index=False)
dim_courses.to_csv(DATA_CLEANED / "dim_courses.csv", index=False)
dim_assessments.to_csv(DATA_CLEANED / "dim_assessments.csv", index=False)
dim_time.to_csv(DATA_CLEANED / "dim_time.csv", index=False)

fact_student_course_performance.to_csv(DATA_CLEANED / "fact_student_course_performance.csv", index=False)
fact_student_course_engagement.to_csv(DATA_CLEANED / "fact_student_course_engagement.csv", index=False)
fact_student_course.to_csv(DATA_CLEANED / "fact_student_course.csv", index=False)

## 11) Data Dictionary – Generate & save (Excel)

In [49]:
def infer_dtype(series: pd.Series) -> str:
    dt = str(series.dtype)
    if "Int" in dt or dt.startswith("int"):
        return "Integer"
    if dt.startswith("float"):
        return "Float"
    if dt == "bool":
        return "Boolean"
    return "String"

descriptions = {
    # dim_students
    ("dim_students", "id_student"): "Unique student identifier (key).",
    ("dim_students", "gender"): "Student gender.",
    ("dim_students", "region"): "Student geographic region.",
    ("dim_students", "highest_education"): "Highest education level before enrollment.",
    ("dim_students", "imd_band"): "Socio-economic deprivation band (IMD).",
    ("dim_students", "age_band"): "Student age group band.",
    ("dim_students", "disability"): "Disability indicator.",
    ("dim_students", "studied_credits"): "Maximum credits studied by student.",
    ("dim_students", "num_of_prev_attempts"): "Maximum number of previous attempts by student.",
    ("dim_students", "final_result"): "Final outcome (e.g., Pass/Fail/Withdrawn).",

    # dim_courses
    ("dim_courses", "module_code"): "Module (course) code.",
    ("dim_courses", "presentation"): "Course presentation/session identifier.",
    ("dim_courses", "module_presentation_length"): "Course duration in days.",
    ("dim_courses", "course_key"): "Surrogate course offering key (module_code_presentation).",

    # dim_assessments
    ("dim_assessments", "id_assessment"): "Assessment identifier (key).",
    ("dim_assessments", "assessment_type"): "Assessment type (TMA/CMA/Exam).",
    ("dim_assessments", "date"): "Assessment due day index (days since course start).",
    ("dim_assessments", "weight"): "Assessment weight (%) in course grading.",
    ("dim_assessments", "course_key"): "Course offering key for this assessment.",
    ("dim_assessments", "module_code"): "Module code.",
    ("dim_assessments", "presentation"): "Presentation/session.",

    # dim_time
    ("dim_time", "day_index"): "Day index (days since course start).",
    ("dim_time", "week_index"): "Week index derived from day_index.",
    ("dim_time", "month_index_approx"): "Approximate month index derived from day_index.",

    # facts
    ("fact_student_course_performance", "avg_score"): "Average raw score across assessments for a student in a course.",
    ("fact_student_course_performance", "assessments_taken"): "Number of assessments submitted by student in a course.",
    ("fact_student_course_performance", "total_weight"): "Total weight of assessments included for student-course.",
    ("fact_student_course_performance", "weighted_points"): "Sum(score * weight) for student-course.",
    ("fact_student_course_performance", "weighted_avg_score"): "Weighted average score for student-course.",
    ("fact_student_course_performance", "pass_flag"): "True if weighted_avg_score >= 50.",
    ("fact_student_course_performance", "at_risk_flag"): "Simple at-risk indicator based on score/participation.",
    ("fact_student_course_performance", "last_submission_day"): "Last assessment submission day index.",

    ("fact_student_course_engagement", "total_clicks"): "Total LMS clicks for a student in a course.",
    ("fact_student_course_engagement", "active_days"): "Number of distinct days with LMS activity for a student-course.",
    ("fact_student_course_engagement", "first_activity_day"): "First LMS activity day index.",
    ("fact_student_course_engagement", "last_activity_day"): "Last LMS activity day index.",
    ("fact_student_course_engagement", "engagement_score"): "Composite engagement score based on clicks and active days.",

    ("fact_student_course", "module_presentation_length"): "Course duration in days (from dim_courses).",
}

def build_dictionary(tables: dict[str, pd.DataFrame]) -> pd.DataFrame:
    rows = []
    for tname, df in tables.items():
        for col in df.columns:
            rows.append({
                "table": tname,
                "column": col,
                "type": infer_dtype(df[col]),
                "description": descriptions.get((tname, col), "—"),
                "example_value": (str(df[col].dropna().iloc[0]) if df[col].dropna().shape[0] > 0 else "—")
            })
    return pd.DataFrame(rows)

tables_for_dict = {
    "dim_students": dim_students,
    "dim_courses": dim_courses,
    "dim_assessments": dim_assessments,
    "dim_time": dim_time,
    "fact_student_course_performance": fact_student_course_performance,
    "fact_student_course_engagement": fact_student_course_engagement,
    "fact_student_course": fact_student_course,
}

data_dictionary_df = build_dictionary(tables_for_dict)

dict_path = DATA_DICT / "data_dictionary_oulad.xlsx"
with pd.ExcelWriter(dict_path, engine="openpyxl") as writer:
    data_dictionary_df.to_excel(writer, sheet_name="DataDictionary", index=False)